# Tarea 2
Estudiantes:
- Matías Fuentes
- Larry Uribne

Carga de librerías e inicialización de sparkContext

In [1]:
from pyspark.sql import SparkSession
import math
spark = SparkSession.builder \
    .getOrCreate()
sc = spark.sparkContext

23/06/21 13:16:49 WARN Utils: Your hostname, Matiass-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.100.162 instead (on interface en0)
23/06/21 13:16:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/21 13:16:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Ejemplo inicial

In [2]:
nodes = [1, 2, 3, 4]
edges = [(1, 2), (2, 3), (2, 4), (3, 2)]
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)

## Problema 1

1. Prepara un RDD que tenga cada nodo con su Page Rank inicial. Luego, haz una función que prepare el
mensaje que cada nodo va a enviar. Probablemente quieras almacenar estos valores como otro RDD.

Preparemos un RDD que contenga a cada nodo con su Page Rank inicial. El Page Rank inicial está dado por $\frac{1}{N_{nodos}}$

In [18]:
n_nodes = rdd_nodes.count()
# Disponibilizamos la cantidad de nodos para todos los workers
bc_n_nodes = sc.broadcast(n_nodes)

rdd_ini = rdd_nodes.map(lambda x: (x, 1/bc_n_nodes.value))
rdd_ini.collect()

[(1, 0.25), (2, 0.25), (3, 0.25), (4, 0.25)]

Para el paso siguiente necesitaremos la cantidad de vecinos

In [19]:
neigh_counts = rdd_edges.countByKey()
# Disponibilizamos la cantidad de mensajes enviados por nodo para todos los workers
bc_neigh_counts = sc.broadcast(neigh_counts) 
bc_neigh_counts.value

defaultdict(int, {1: 1, 2: 2, 3: 1})

El mensaje saliente de cada nodo será el mensaje inicial dividido por la cantidad de vecinos. Vemos que solo el nodo 2 tiene más de 1 vecino, por tanto el único mensaje que será modificado es el suyo.

In [78]:
def prepare_node_msg(node, init_msg):
    if neigh_counts[node] != 0:
        message = init_msg / bc_neigh_counts.value[node]
        return message
    return init_msg

def preprare_rdd_msg(rdd_ini):
    rdd_prep_msg = rdd_ini.map(lambda x: (x[0], prepare_node_msg(x[0], x[1])))
    return rdd_prep_msg

rdd_prep_msg = preprare_rdd_msg(rdd_ini)
rdd_prep_msg.collect()

[(1, 0.25), (2, 0.125), (3, 0.25), (4, 0.25)]

2. Función que envía los mensajes a los nodos correspondientes y se hace cargo del merge de los mensajes recibidos por cada nodo. Debe retornar un RDD que para cada nodo diga cuál es el mensaje final recibido.

Priemro hacemos un join. Las llaves son los nodos emisores, y os valores resultantes seran tuplas donde el primer elemento es el nodo receptor y el segundo elemento es el mensaje recibido.

Luego, generamos otro RDD que contenga solo los valores de la operación anterior y aplicamos reduceByKey para sumar los mensajes, obteniendo así tuplas donde la llave es el nodo receptor y el valor es la suma de todos los mensajes recibidos.

In [27]:
def send_msg(rdd_edges, rdd_prep_msg):
    rdd_msg_sent = rdd_edges.join(rdd_prep_msg)
    rdd_received_msg = rdd_msg_sent.values().reduceByKey(lambda x, y: x + y)
    return rdd_received_msg
rdd_received_msg = send_msg(rdd_edges, rdd_prep_msg)
rdd_received_msg.collect()

[(2, 0.5), (3, 0.125), (4, 0.125)]

3. Haz una función que actualice el valor de Page Rank para cada nodo considerando el damping factor.
Probablemente quieras hacer una función que tome el output del punto anterior y lo procese.

In [13]:
# Disponibilizamos el damping factor para todos los workers
damping = 0.85
bc_damping = sc.broadcast(damping)
def update_pr(rdd_received_msg):
    rdd_updated_pr = rdd_received_msg.mapValues(lambda x: x * bc_damping.value + (1-bc_damping.value)/bc_n_nodes.value)
    return rdd_updated_pr
rdd_updated_pr = update_pr(rdd_received_msg)
rdd_updated_pr.collect()

[(2, 0.4625), (3, 0.14375), (4, 0.14375)]

4. Itera los pasos correspondientes por un número máximo de iteraciones, o hasta que la diferencia entre dos iteraciones del valor de Page Rank sea mínima.

Para facilitar la legibilidad, consolidemos el proceso completo en una función. Usaremos el error absoluto medio como criterio de parada 

In [79]:
def mean_absolute_error(rdd1, rdd2):
    abs_difference = rdd1.join(rdd2).values().map(lambda x: abs(x[0] - x[1]))
    mean_abs_difference = abs_difference.reduce(lambda x, y: x + y)/2
    return mean_abs_difference

def page_rank(rdd_nodes, rdd_edges, damping=0.85, max_iterations = 10, eps=0.05):
    n_nodes = rdd_nodes.count()
    # Disponibilizamos la cantidad de nodos para todos los workers
    bc_n_nodes = sc.broadcast(n_nodes)
    neigh_counts = rdd_edges.countByKey()
    # Disponibilizamos la cantidad de mensajes enviados por nodo para todos los workers
    bc_neigh_counts = sc.broadcast(neigh_counts) 
    bc_neigh_counts.value
    # Disponibilizamos el damping factor para todos los workers
    bc_damping = sc.broadcast(damping)
    # PageRank inicial
    rdd_ini = rdd_nodes.map(lambda x: (x, 1/bc_n_nodes.value))
    prev_pr = rdd_ini
    # Preparar mensaje inicial
    rdd_prep_msg = preprare_rdd_msg(rdd_ini)
    
    for i in range(max_iterations):
        # Enviar mensaje y gestionar merge al recibir
        rdd_received_msg = send_msg(rdd_edges, rdd_prep_msg)
        # Actualizar PageRank
        rdd_updated_pr = update_pr(rdd_received_msg)
        # Calcular distancia entre el PageRank recién obtenido y el de la iter previa
        mean_distance = mean_absolute_error(prev_pr, rdd_updated_pr)
        # Condición de parada
        if mean_distance < eps:
            break
        # Mensaje enviado para iteración siguiente
        rdd_prep_msg = preprare_rdd_msg(rdd_updated_pr)
        # El PageRank de ahora es el PageRank previo en la próxima iteración
        prev_pr = rdd_updated_pr
        
    print(f'Total iterations: {i}')
    return rdd_updated_pr

In [81]:
final_pr = page_rank(rdd_nodes, rdd_edges)
final_pr.collect()

Total iterations: 5


[(2, 0.11527618701171877), (3, 0.10328731884765627), (4, 0.10328731884765627)]

### Generando grafos de prueba

In [84]:
import numpy as np

In [82]:
# def generate_graph(n_nodes, n_edges, tier_dict):
#     nodes = list(range(1, n_nodes+1))
#     uniform_prob = 1/n_nodes
#     probabilities = [0.2, 0.5, 0.3]
#     np.random.choice(elements, 10, p=probabilities)
#     return nodes


## Problema 2

Recibir input

In [101]:
# Ejemplo tomado de https://www.geeksforgeeks.org/dijkstras-shortest-path-algorithm-greedy-algo-7/
nodes = list(range(1, 9))
edges = [(1,2,4), (1,3,2), (1,4,1), (3,2,1), (3,5,6),(4,5,9),(4,6,2),(7,8,9), (5,6,1)]

Generar RDDs

In [104]:
initial_node = 1
bc_initial_node = sc.broadcast(initial_node) # Disponibilizar para todos los worker
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)

Formateamos las aristas

In [116]:
rdd_edges = rdd_edges.map(lambda x: (x[0], (x[1], x[2])))
rdd_edges.collect()

[(1, (2, 4)),
 (1, (3, 2)),
 (1, (4, 1)),
 (3, (2, 1)),
 (3, (5, 6)),
 (4, (5, 9)),
 (4, (6, 2)),
 (7, (8, 9)),
 (5, (6, 1))]

1. Escoge el nodo inicial, este nodo tiene costo acumulado 0 y todos los demás tienen costo acumulado
infinito.

In [117]:
rdd_init = rdd_nodes.map(lambda x: (x, math.inf) if x != bc_initial_node.value else (x, 0))
rdd_init.collect()

[(1, 0), (2, inf), (3, inf), (4, inf), (5, inf), (6, inf), (7, inf), (8, inf)]

2. En cada iteración, cada nodo comunica el costo acumulado a sus vecinos. Cada nodo recibe este costo,
sumado con el costo de atravesar la arista.

In [122]:
rdd_edges.join(rdd_init).collect()

[(1, ((2, 4), 0)),
 (1, ((3, 2), 0)),
 (1, ((4, 1), 0)),
 (3, ((2, 1), inf)),
 (3, ((5, 6), inf)),
 (4, ((5, 9), inf)),
 (4, ((6, 2), inf)),
 (5, ((6, 1), inf)),
 (7, ((8, 9), inf))]

In [125]:
rdd_pass_msg = rdd_edges.join(rdd_init).mapValues(lambda x: (x[0][0], x[0][1] + x[1])).values()
rdd_pass_msg.collect()

[(2, 4),
 (3, 2),
 (4, 1),
 (2, inf),
 (5, inf),
 (5, inf),
 (6, inf),
 (6, inf),
 (8, inf)]

3. Para hacer merge de todos los mensajes dejamos el mínimo de todos los costos. Así, actualizamos cada
nodo con el costo mínimo recibido solo si es menor al costo acumulado que ya tenía ese nodo.

In [128]:
rdd_reduce_msg = rdd_pass_msg.reduceByKey(lambda x, y: min(x, y))
rdd_reduce_msg.collect()

[(2, 4), (3, 2), (4, 1), (5, inf), (6, inf), (8, inf)]

In [139]:
def robust_min(x):
    a, b = x
    try:
        return min(a,b)
    except:
        return a if a != None else b
    
rdd_updated_cost = rdd_init.leftOuterJoin(rdd_reduce_msg).mapValues(lambda x: robust_min(x))
rdd_updated_cost.collect()

[(1, 0), (2, 4), (3, 2), (4, 1), (5, inf), (6, inf), (7, inf), (8, inf)]

4. Si en dos iteraciones el costo en llegar para cada nodo no cambia, entonces nos detenemos.

In [166]:
def stop_condition(rdd_pre, rdd_post):
    # RDD donde las llaves son los nodos y los valores son 1 si no hubo cambio y 0 si lo hubo
    rdd_track_changes = rdd_pre.join(rdd_post).mapValues(lambda x: int(x[0] == x[1]))
    # Intersección
    n_changes = rdd_track_changes.values().reduce(lambda x, y: x + y) 
    return n_changes == 0

In [167]:
stop_condition(rdd_init, rdd_updated_cost)

False